In [146]:
from census import Census
from us import states
import pandas as pd
import numpy as np
import censusdata
import requests
import us
import matplotlib.pyplot as plt
from keys import api_census_key

c2016 = Census(api_census_key, year = 2016)


In [147]:
#Access census database for 2016
census_data2016 = c2016.acs5.get(('NAME', 'B01003_001E'),
         {'for': 'state:{}'.format(Census.ALL)})

In [148]:
#make dataframe
pop_2016 = pd.DataFrame(census_data2016)

#obtain population
pop_2016 = pop_2016.rename(columns={"B01003_001E": "2016 Population",
                                   "NAME": "City"})

pop_2016.head()

#change in population from 2014 to 2017
#a chart

,2016 Population,City,state
0,4841164.0,Alabama,01
1,736855.0,Alaska,02
2,6728577.0,Arizona,04
3,2968472.0,Arkansas,05
4,38654206.0,California,06


In [149]:
#request 2017 census data
url_2017 = "https://api.census.gov/data/2017/pep/population?get=POP,GEONAME&for=state:*&DATE=9"
request_2017 = requests.get(url_2017).json()

#create dataframe
request_2017_df = pd.DataFrame(request_2017,columns=request_2017[0])

#obtain desired data
pop_2017 = request_2017_df[['POP','GEONAME']]
pop_2017 = pop_2017.rename(columns = {"POP":" 2017 Population",
                                     "GEONAME":"State"})
pop_2017



,2017 Population,State
0,POP,GEONAME
1,4860545,Alabama
2,741522,Alaska
3,6908642,Arizona
4,2988231,Arkansas
5,39296476,California
6,5530105,Colorado
7,3587685,Connecticut
8,952698,Delaware
9,684336,District of Columbia


In [150]:
dfholder = []
years = [2016,2015,2014,2013]
for year in years:
    census_data = Census(api_census_key, year = year)
    census_data_years = census_data.acs5.get(('NAME', 'B01003_001E'),
                                             {'for': 'state:{}'.format(Census.ALL)})
    pop_census = pd.DataFrame(census_data_years)
    
    pop_census = pop_census.rename(columns = {"B01003_001E": f"{year} Population",
                                             "NAME":"State",
                                             "state":"State Number"})
    dfholder.append(pop_census)


In [151]:
merge_census = pop_2017.merge(dfholder[0]).merge(dfholder[1]).merge(dfholder[2]).merge(dfholder[3]).set_index(['State','State Number'])

merge_census

,,2017 Population,2016 Population,2015 Population,2014 Population,2013 Population
State,State Number,,,,,
Alabama,01,4860545,4841164.0,4830620.0,4817678.0,4799277.0
Alaska,02,741522,736855.0,733375.0,728300.0,720316.0
Arizona,04,6908642,6728577.0,6641928.0,6561516.0,6479703.0
Arkansas,05,2988231,2968472.0,2958208.0,2947036.0,2933369.0
California,06,39296476,38654206.0,38421464.0,38066920.0,37659181.0
Colorado,08,5530105,5359295.0,5278906.0,5197580.0,5119329.0
Connecticut,09,3587685,3588570.0,3593222.0,3592053.0,3583561.0
Delaware,10,952698,934695.0,926454.0,917060.0,908446.0
District of Columbia,11,684336,659009.0,647484.0,633736.0,619371.0
